# CatSniffer Setup and Firmware Strategy

CatSniffer is a flexible wireless research platform. Its power comes from **task-specific firmware**: 
the same hardware can act as a sniffer, a fuzzer, or an HCI bridge depending on the binary you load. 
This notebook is your control panel for managing firmware and running experiments directly from Python.

### Why task-focused firmware?

Radios have strict timing requirements, and each activity places different demands on the device. 
A single "do-everything" firmware would be unreliable. Instead, we provide lean images for each purpose:

- **Sniffer firmware** → For capturing packets with accurate timestamps and exporting PCAPs for Wireshark or analysis.  
- **Fuzzer firmware** → For generating and sending malformed frames, controlling mutation strategies, and stress-testing targets.  
- **HCI-only firmware** → For experiments where the notebook acts as the Bluetooth Host, sending raw HCI commands directly.  
- **Replay firmware** → For reproducing captured traffic, validating responses, or simulating known devices.  

By selecting the right firmware, you ensure your CatSniffer performs optimally for the task at hand.

### How this notebook is organized

Each section of this notebook walks you through a specific step: checking prerequisites, selecting firmware, 
flashing the device, and running experiments. Before every code cell, you’ll find a short explanation of what it does 
so you always know what to expect.



### Clone or update Catsniffer-Tools

This code cell will execute part of the CatSniffer setup process. It checks out or updates the CatSniffer repository containing firmware and utilities.

**Expected output:** You should see messages indicating whether the repository was cloned or updated.

In [1]:
import os
import subprocess

repo_url = "https://github.com/ElectronicCats/CatSniffer-Tools.git"
target_dir = "CatSniffer-Tools"

if not os.path.exists(target_dir):
    print(f"Cloning repository {repo_url}...")
    !git clone {repo_url}
else:
    print(f"Updating existing repository in {target_dir}...")
    try:
        subprocess.run(["git", "-C", target_dir, "pull"], check=True)
        print("Repository updated successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Failed to update repository: {e}")

Updating existing repository in CatSniffer-Tools...
Already up to date.
Repository updated successfully.


### Install dependencies

This code cell will execute part of the CatSniffer setup process. It installs the required python packages using pip.

**Expected output:** The libraries should be installed.

In [2]:
# Install dependencies
import sys
!{sys.executable} -m pip install "typer[all]" pyserial intelhex requests ipywidgets

### Connect your Catsniffer and identify the USB serial port assigned

This code cell will identify and select the serial port used by the Catsniffer.

**Expected output:** The cell will confirm the serial port is identified.

In [3]:
# Automatically detect your USB serial port
import serial.tools.list_ports

def detect_serial_port():
    ports = serial.tools.list_ports.comports()
    for port in ports:
        if any(x in port.device for x in ["ttyUSB", "ttyACM", "cu.usbmodem", "usbserial", "COM"]):
            return port.device
    return None

selected_port = detect_serial_port()
if selected_port:
    print(f"Serial Port identified: {selected_port}")
else:
    print("No serial port detected.")
    

Serial Port identified: /dev/cu.usbmodem11101


### Let's try some of the official firmwares distributed via catnip_uploader

This code cell will execute part of the CatSniffer setup process. It shows a menu and flashes the selected firmware binary onto your CatSniffer device.

**Expected output:** Look for 'Firmware uploaded successfully.' or similar confirmation in the output.

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess
import sys
import os

uploader_path = "CatSniffer-Tools/catnip_uploader/catnip_uploader.py"
selected_port = selected_port if 'selected_port' in globals() else '/dev/cu.usbmodem2101'

# 📦 Function to parse the firmware table
def parse_firmware_table(lines):
    firmwares = []
    current_firmware = None
    current_description = ""

    for line in lines:
        if line.strip().startswith('│'):
            parts = line.split('│')
            if len(parts) >= 3:
                firmware = parts[1].strip().split('_cc')[0]
                description = parts[2].strip()
                if firmware:
                    if current_firmware:
                        firmwares.append((current_firmware, current_description.strip()))
                    current_firmware = firmware
                    current_description = description
                else:
                    current_description += " " + description
    if current_firmware:
        firmwares.append((current_firmware, current_description.strip()))
    return firmwares

# 🧪 Load firmwares with subprocess
def get_firmwares():
    try:
        result = subprocess.run(
            [sys.executable, uploader_path, "releases"],
            capture_output=True,
            text=True,
            check=True
        )
        return parse_firmware_table(result.stdout.splitlines())
    except subprocess.CalledProcessError as e:
        print("❌ Error obtaining the firmwares:")
        print(e.stderr)
        return []

# 🌐 Dropdown to show available firmwares
firmware_list = get_firmwares()
firmware_options = [(f"{name} - {desc}", name) for name, desc in firmware_list]

firmware_dropdown = widgets.Dropdown(
    options=firmware_options,
    description='Firmware:',
    layout=widgets.Layout(width='80%')
)

# 🔘 Load button
load_button = widgets.Button(description="Load Firmware", button_style='success')
output_load = widgets.Output()

def on_load_clicked(b):
    selected = firmware_dropdown.value
    if selected and selected_port:
        with output_load:
            clear_output()
            print(f"Loading firmware '{selected}' to the serial port {selected_port}...")
        try:
            result = subprocess.run(
                [sys.executable, uploader_path, "load", selected, selected_port, "--validate"],
                capture_output=True,
                text=True,
                check=True
            )
            with output_load:
                print(result.stdout)
        except subprocess.CalledProcessError as e:
            with output_load:
                print("❌ Error while loading the firmware:")
                print("\n--- STDOUT ---\n", e.stdout)
                print("\n--- STDERR ---\n", e.stderr)
    else:
        with output_load:
            print("⚠️ No firmware or port selected.")

load_button.on_click(on_load_clicked)

# 🎛️ Display UI
display(firmware_dropdown, load_button, output_load)


Dropdown(description='Firmware:', layout=Layout(width='80%'), options=(('sniffle - BLE sniffer for Bluetooth 5…

Button(button_style='success', description='Load Firmware', style=ButtonStyle())

Output()

You should see the message 'Firmware uploaded successfully.' in the output after selecting and loading a firmware.

### TI Sniffer

Let's try installing and testing the TI sniffer first:
* Select 'sniffer_fw' from the firmware list and load it into the catsniffer in the previous step
* Click the button 'open serial port' to connect to your usb serial port at 115200 bauds.
* Send any data using the Command field.
* See the returned messages. The message 'TI Packet Sniffer ver.1.10.0' should be printed several times.

**Expected output:** Look for the message 'TI Packet Sniffer ver.1.10.0' in the output. The message indicates that the new firmware is running correctly.

In [9]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import serial, time, threading

serial_conn = None
serial_port = selected_port if 'selected_port' in globals() else '/dev/cu.usbmodem2101'

output_serial = widgets.Output()
command_input = widgets.Text(description='Command:')
send_button = widgets.Button(description='Send')
close_button = widgets.Button(description='Close Serial Port', button_style='danger')
open_button = widgets.Button(description='Open Serial Port', button_style='success')

def open_serial(b=None):
    global serial_conn
    try:
        serial_conn = serial.Serial(serial_port, 115200, timeout=1)
        time.sleep(2)
        with output_serial:
            clear_output()
            print(f'✅ Port {serial_port} opened at 115200 bauds.')
    except Exception as e:
        with output_serial:
            clear_output()
            print(f'❌ Error while opening the serial port: {e}')

def close_serial(b=None):
    global serial_conn
    if serial_conn and serial_conn.is_open:
        serial_conn.close()
        with output_serial:
            print('🔌 Serial Port closed correctly.')

def send_command(b):
    global serial_conn
    if serial_conn and serial_conn.is_open:
        cmd = command_input.value.strip() + '\r\n'
        serial_conn.write(cmd.encode())
        time.sleep(0.3)
        resp = serial_conn.read_all().decode(errors='ignore')
        with output_serial:
            print(f'📤 Enviado: {cmd.strip()}')
            print(f'📥 Respuesta: {resp.strip()}')
    else:
        with output_serial:
            print('⚠️ The serial port is not opened.')

# Button callbacks
send_button.on_click(send_command)
close_button.on_click(close_serial)
open_button.on_click(open_serial)

# Display UI
display(widgets.HBox([open_button, close_button]))
display(command_input, send_button, output_serial)


Text(value='', description='Command:')

Button(description='Send', style=ButtonStyle())

Output()